# 建立訓練資料

## 初始環境設定
- 切換到 LLama-factory image kernel

In [1]:
# 初始環境設定
import os
from pathlib import Path
HOME = str(Path.home())
Add_Binarry_Path=HOME+'/.local/bin:/usr/ubuntu_bin'
os.environ['PATH']=os.environ['PATH']+':'+Add_Binarry_Path

In [2]:
!nvidia-smi
import torch
torch.cuda.is_available()

Mon Mar 11 19:29:44 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:1B:00.0 Off |                    0 |
| N/A   33C    P0    41W / 300W |      3MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  Off  | 00000000:3D:00.0 Off |                    0 |
| N/A   

True

## 套件安裝 OPENCC
```
s2t.json Simplified Chinese to Traditional Chinese 簡體到繁體
s2tw.json Simplified Chinese to Traditional Chinese (Taiwan Standard) 簡體到臺灣正體
```

In [ ]:
# 映像檔已經安裝
#!pip install opencc
#!pip install opencc-python-reimplemented

## DATASET

In [33]:
import json
from datasets import load_dataset
import opencc #繁体简体互转
#op_cc=opencc.OpenCC('s2t')
op_cc=opencc.OpenCC('s2twp')

# 讀取數據集，take可以取得該數據集前n筆資料
#dataset = load_dataset("Hello-SimpleAI/HC3-Chinese", split="train", streaming=True,encoding='utf-8')
dataset_dict = load_dataset("Hello-SimpleAI/HC3-Chinese", name="baike")
dataset = dataset_dict['train']

# 提取所需欄位並建立新的字典列表
limit=0
extracted_data = []
for example in dataset:
    extracted_example = {
        "instruction": op_cc.convert("你是個知識豐富的人工智慧助手，使用者將以中文向你提問，你將根據你的知識用中文來如實回答問題"),
        "input":  op_cc.convert(example["question"]),
        #"output":  op_cc.convert(example["human_answers"]),
        "output":  op_cc.convert(example["chatgpt_answers"][0])        
    }
    extracted_data.append(extracted_example)
    if len(extracted_data) == limit:
        break

# 指定 JSON 文件名稱
json_filename = "data.json"

# 寫入 JSON 文件
with open(json_filename, "w") as json_file:
    json.dump(extracted_data, json_file, indent=4)

print(f"數據已提取並保存為 {json_filename}")

數據已提取並保存為 data.json


In [40]:
# 所有資料內容
import pandas as pd
df = pd. read_json ( 'data.json' )
#df.describe()
df

,instruction,input,output
0,你是個知識豐富的人工智慧助手，使用者將以中文向你提問，你將根據你的知識用中文來如實回答問題,我有一個計算機相關的問題，請用中文回答，什麼是 硬碟安裝,硬碟安裝是指將軟體或作業系統安裝到硬碟上的過程。硬碟是電腦的主儲存器，通常使用硬碟來安裝作業...
1,你是個知識豐富的人工智慧助手，使用者將以中文向你提問，你將根據你的知識用中文來如實回答問題,我有一個計算機相關的問題，請用中文回答，什麼是 網際協議,"網際協議 (Internet Protocol, IP) 是一種用於計算機之間在計算機網路中..."
2,你是個知識豐富的人工智慧助手，使用者將以中文向你提問，你將根據你的知識用中文來如實回答問題,我有一個計算機相關的問題，請用中文回答，什麼是 外存,外存是計算機中用來儲存資料和程式的裝置，其中儲存的資料和程式在計算機斷電後也不會丟失。常見的...
3,你是個知識豐富的人工智慧助手，使用者將以中文向你提問，你將根據你的知識用中文來如實回答問題,我有一個計算機相關的問題，請用中文回答，什麼是 程式流程圖,程式流程圖是用圖形表示程式的邏輯流程和執行順序的圖表。它使用各種圖形和符號，如箭頭、框、圓圈...
4,你是個知識豐富的人工智慧助手，使用者將以中文向你提問，你將根據你的知識用中文來如實回答問題,我有一個計算機相關的問題，請用中文回答，什麼是 控制匯流排,控制匯流排是計算機中一種用於傳輸控制資訊的匯流排。它連線著計算機的處理器、記憶體和其他裝置，...
...,...,...,...
4612,你是個知識豐富的人工智慧助手，使用者將以中文向你提問，你將根據你的知識用中文來如實回答問題,我有一個資訊科學相關的問題，請用中文回答，什麼是 奈米MOSFET,奈米MOSFET（Metal-Oxide-Semiconductor Field-Effec...
4613,你是個知識豐富的人工智慧助手，使用者將以中文向你提問，你將根據你的知識用中文來如實回答問題,我有一個資訊科學相關的問題，請用中文回答，什麼是 IP隧道,IP隧道是一種網路技術，它允許在兩個不能直接連線的網路之間建立一條隱藏的通道。這條隱藏的通道...
4614,你是個知識豐富的人工智慧助手，使用者將以中文向你提問，你將根據你的知識用中文來如實回答問題,我有一個資訊科學相關的問題，請用中文回答，什麼是 鄰通道,鄰通道是資訊科學中的一個術語，它指的是在訊號傳輸過程中，訊號的不同部分之間的相互關係。換句話...
4615,你是個知識豐富的人工智慧助手，使用者將以中文向你提問，你將根據你的知識用中文來如實回答問題,我有一個資訊科學相關的問題，請用中文回答，什麼是 zswap,zswap 是一種 Linux 核心的記憶體壓縮方式。它將系統記憶體中的空閒頁面壓縮後儲存在...


In [37]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="data.json", split="train")
print(dataset)
print(dataset[0])

Dataset({
    features: ['input', 'output', 'instruction'],
    num_rows: 4617
})
{'input': '我有一個計算機相關的問題，請用中文回答，什麼是 硬碟安裝', 'output': '硬碟安裝是指將軟體或作業系統安裝到硬碟上的過程。硬碟是電腦的主儲存器，通常使用硬碟來安裝作業系統和其他軟體。在安裝硬碟之前，你需要準備好安裝介質（通常是光碟或 USB 快閃記憶體盤），並按照作業系統的安裝嚮導進行操作。安裝完成後，你就可以使用硬碟上的軟體了。', 'instruction': '你是個知識豐富的人工智慧助手，使用者將以中文向你提問，你將根據你的知識用中文來如實回答問題'}


## DATASET

In [ ]:
import json
from datasets import load_dataset
import opencc #繁体简体互转
#op_cc=opencc.OpenCC('s2t')
op_cc=opencc.OpenCC('s2twp')

# 讀取數據集，take可以取得該數據集前n筆資料
dataset = load_dataset("michaelwzhu/ChatMed_Consult_Dataset", split="train", streaming=True,encoding='utf-8')

# 提取所需欄位並建立新的字典列表
limit=0
extracted_data = []
for example in dataset:
    extracted_example = {
        "instruction": op_cc.convert("现在你是一名专业的中医医生，请用你的专业知识提供详尽而清晰的关于中医问题的回答。"),
        "input":  op_cc.convert(example["query"]),
        "output":  op_cc.convert(example["response"])
    }
    extracted_data.append(extracted_example)
    if len(extracted_data) == limit:
        break

# 指定 JSON 文件名稱
json_filename = "data.json"

# 寫入 JSON 文件
with open(json_filename, "w") as json_file:
    json.dump(extracted_data, json_file, indent=4)

print(f"數據已提取並保存為 {json_filename}")

In [4]:
# 所有資料內容
import pandas as pd
df = pd. read_json ( 'data.json' )
df.describe()

,instruction,input,output
count,549326,549326,549326
unique,1,544995,549286
top,現在你是一名專業的中醫醫生，請用你的專業知識提供詳盡而清晰的關於中醫問題的回答。,平時無不適，昨天晚上聚餐，喝了些酒，有點高了；早上感覺有些頭暈，肚子也難受，上了幾次廁所，便...,我不太明白您的意思，請問您需要什麼幫助嗎？
freq,549326,26,5


In [5]:
# 取出前一萬筆, 並儲存
import pandas as pd
df = pd. read_json ( 'data.json' )
dataset_df_10k = df[:10000]
dataset_df_10k.to_json('LLaMA-Factory/data/medicalQA_10k.json', orient='records')
dataset_df_10k

,instruction,input,output
0,現在你是一名專業的中醫醫生，請用你的專業知識提供詳盡而清晰的關於中醫問題的回答。,這段時間去上廁所本來想小便的可是每次都會拉大便,這可能是因為你的飲食習慣或者消化系統的問題導致的。建議你試著調整一下飲食，增加膳食纖維的攝入...
1,現在你是一名專業的中醫醫生，請用你的專業知識提供詳盡而清晰的關於中醫問題的回答。,醫生呀！我剛被查出得了白癜風，不知道初期治療需要多少錢？,白癜風的治療費用因個體差異和治療方案的不同而有所差異。初期治療主要以口服藥物和外用藥物為主，...
2,現在你是一名專業的中醫醫生，請用你的專業知識提供詳盡而清晰的關於中醫問題的回答。,男朋友31歲，前列腺B超資料是34*48*31，，包膜完整平滑，內部回聲均勻，未見明顯異常回...,根據您提供的資料，男朋友的前列腺B超結果表明前列腺肥大增生，但是並未出現明顯異常回聲，所以情...
3,現在你是一名專業的中醫醫生，請用你的專業知識提供詳盡而清晰的關於中醫問題的回答。,左大腿脾經內側疼痛大概有6寸左右，有幾天了，不知道怎麼導致的，表面沒什麼就是一按就刺痛，這怎麼回事,左大腿內側疼痛可能是由多種因素引起的，例如：\n\n1. 肌肉勞損或拉傷：如果您最近進行了劇...
4,現在你是一名專業的中醫醫生，請用你的專業知識提供詳盡而清晰的關於中醫問題的回答。,你好，全身沒勁，沒精神，吃不下飯，只想睡覺，是什麼情況,這些症狀可能是身體出現了一些問題，建議您去看醫生進行檢查。這些症狀可能和許多疾病有關，比如抑...
...,...,...,...
9995,現在你是一名專業的中醫醫生，請用你的專業知識提供詳盡而清晰的關於中醫問題的回答。,3個月寶寶一個星期才拉一次大便，純母乳餵養。去醫院檢查拍片醫生說是腸子堵塞說是用點開塞露。不...,對於純母乳餵養的寶寶來說，每天排便次數和大便的性狀可能會有所不同，但是長時間沒有大便可能會引...
9996,現在你是一名專業的中醫醫生，請用你的專業知識提供詳盡而清晰的關於中醫問題的回答。,現在血糖高，37周，孩子臍帶繞頸一週，胎動減弱。有宮縮，我該怎麼辦,如果你有宮縮和減弱的胎動，建議你立即就醫。你可以聯絡你的產科醫生或撥打急救電話。如果你的血糖...
9997,現在你是一名專業的中醫醫生，請用你的專業知識提供詳盡而清晰的關於中醫問題的回答。,你好。醫生，我是右大腿被車撞斷了，上了鋼板了，我已經和我老婆發生了性生活，請問一下，對腿有影...,首先，性生活通常不會對腿的康復產生直接的影響。但是，如果你感到疼痛或不適，那麼最好停止性生活...
9998,現在你是一名專業的中醫醫生，請用你的專業知識提供詳盡而清晰的關於中醫問題的回答。,我的牙齒被蛀牙了一半拔掉了一半自己掉了裡面的肉能不能拔掉,作為智慧助手，我建議您諮詢一下牙醫的意見。如果您的牙齒已經被蛀牙了一半，那麼最好的選擇是去看...


In [8]:
# sha1sum , open LLaMA-Factory/data/dataset_info.json
!sha1sum LLaMA-Factory/data/medicalQA_10k.json

0db55d43479e101e542f4cd719703ece3a65cc9d  LLaMA-Factory/data/medicalQA_10k.json


```
"medical": {
    "file_name": "medicalQA_10k.json",
    "file_sha1": "0db55d43479e101e542f4cd719703ece3a65cc9d  ",
    "columns": {
      "prompt": "instruction",
      "query": "input",
      "response": "output"
    }
  },
```